# $\mathbb{M}_2$ Distance Map
In this notebook we the compute distance map in $\mathbb{M}_2$ with the $\mathbb{M}_2$ cost function, corresponding to Figure 4c in ["Crossing-Preserving Geodesic Tracking on Spherical Images"](https://arxiv.org/abs/2504.03388v1).

In [ ]:
import numpy as np
import taichi as ti
ti.init(arch=ti.gpu, debug=False, device_memory_GB=3.5) # Use less than the VRAM on your device as to not mix RAM and VRAM
import eikivp
from eikivp.M2.vesselness import import_vesselness
from eikivp.utils import cost_function
from eikivp.M2.plus.distancemap import export_W
from copy import deepcopy

## Parameters

In [ ]:
σ_s_list = np.array((0.5**3, 0.5)) # np.array((1.5, 2.))
σ_o = 0.5 * 0.75**2
σ_s_ext = 1.
σ_o_ext = 0.01
image_name = "E46_OD_best"
image_file_name = f"data\{image_name}.tif"
V_params = {
    "σ_s_list": σ_s_list,
    "σ_o": σ_o,
    "σ_s_ext": σ_s_ext,
    "σ_o_ext": σ_o_ext,
    "image_name": image_name 
}

In [ ]:
V = import_vesselness(V_params, "storage\\vesselness")
dim_I, dim_J, dim_K = V.shape

In [ ]:
Is, Js, Ks = np.indices((dim_I, dim_J, dim_K))
a = 13 / 21
c = np.cos(np.pi/3)
x_min, x_max = -0.866, 0.866
y_min, y_max = -0.866, 0.866
θ_min, θ_max = 0., 2 * np.pi
dxy = (x_max - x_min) / (dim_I - 1)
dθ = (θ_max - θ_min) / dim_K
xs, ys, θs = eikivp.M2.utils.coordinate_array_to_real(Is, Js, Ks, x_min, y_min, θ_min, dxy, dθ)

In [ ]:
λ = 500
p = 2
ξ = 4.
source_point_real_W2 = (0.177528, 0.159588, 2.37002)
source_point_real = eikivp.W2.utils.Π_forward_np(*source_point_real_W2, a, c)
source_point = eikivp.M2.utils.coordinate_real_to_array(*source_point_real, x_min, y_min, θ_min, dxy, dθ)
W_params = deepcopy(V_params)
W_params["λ"] = λ
W_params["p"] = p
W_params["ξ"] = ξ
W_params["source_point"] = source_point
W_params["target_point"] = "default"
W_params["cost_domain"] = "M2"

In [ ]:
target_point_real_W2 = (-0.721357, 0.218753, 2.65495)
target_point_real = eikivp.W2.utils.Π_forward_np(*target_point_real_W2, a, c)
target_point = eikivp.M2.utils.coordinate_real_to_array(*target_point_real, x_min, y_min, θ_min, dxy, dθ)

In [ ]:
C = cost_function(V, λ, p)

In [ ]:
fig, ax, _ = eikivp.visualisations.plot_image_array(C.min(-1), x_min, x_max, y_min, y_max)
ax.scatter(*source_point_real[:-1], label="Source")
ax.arrow(*source_point_real[:-1], 0.1 * np.cos(θs[source_point]), 0.1 * np.sin(θs[source_point]), width=0.01)
ax.scatter(*target_point_real[:-1], label="Target")
ax.arrow(*target_point_real[:-1], 0.1 * np.cos(θs[target_point]), 0.1 * np.sin(θs[target_point]), width=0.01)
ax.legend();

## Compute Distance Map

In [ ]:
W, grad_W = eikivp.eikonal_solver_M2_plus(C, source_point, ξ, dxy, dθ, θs, target_point=target_point, n_max=1e4, n_max_initialisation=1e4, n_check=2e3, n_check_initialisation=2e3, tol=1e-3, initial_condition=200.)

In [ ]:
fig, ax, _ = eikivp.visualisations.plot_image_array(C.min(-1), x_min, x_max, y_min, y_max, figsize=(12, 10))
max_distance = W[target_point] * 2.5
_, _, contour = eikivp.visualisations.plot_contour(W.min(-1), xs[..., 0], ys[..., 0], fig=fig, ax=ax, levels=np.linspace(0., max_distance, 5))
ax.scatter(*source_point_real[:-1], label="Source")
ax.scatter(*target_point_real[:-1], label="Target")
ax.set_aspect("equal")
fig.colorbar(contour, label="$\\min_θ W(x, y, θ)$")
ax.legend();

In [ ]:
export_W(W, grad_W, W_params, "storage\\distance")